In [5]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from collections import defaultdict
from spellchecker import SpellChecker
from kedro.pipeline.decorators import log_time

In [3]:
reviews = io.load('reviews_master')

2019-06-15 23:22:56,130 - kedro.io.data_catalog - INFO - Loading data from `reviews_master` (PickleLocalDataSet)...


In [29]:
# Strip punctuation

def _remove_punc(text):
    return re.sub(r'[^\w\s]','', text)

porter = nltk.PorterStemmer() 

spell = SpellChecker()

def _remove_extensions(lst):
    pattern = re.compile(r"(.)\1{2,}")
    return [pattern.sub(r"\1", w) for w in lst]

def  _fix_spelling(lst):
    return [spell.correction(x) for x in lst]

def _stem_words(lst):
    return [porter.stem(x) for x in lst]

stop_words = set(stopwords.words('english'))
stop_words = [_remove_punc(x) for x in stop_words]

def _remove_stops(lst):
    return [x for x in lst if x not in stop_words]

def _count_words(lst):
    word_freq = defaultdict(int)
    for w in lst:
        word_freq[w] += 1
    return word_freq

def create_bag(text):
    text = _remove_punc(text)
    text = text.lower()
    words = nltk.word_tokenize(text)
#     words = _remove_extensions(words)
#     words = _fix_spelling(words)
    words = _remove_stops(words)
    freq = _count_words(words)
    return freq

In [31]:
corpus = reviews['review_text']
corpus.apply(create_bag)

id
0        {'absolutely': 1, 'wonderful': 1, 'silky': 1, ...
1        {'love': 2, 'dress': 1, 'sooo': 1, 'pretty': 1...
2        {'high': 1, 'hopes': 1, 'dress': 1, 'really': ...
3        {'love': 3, 'jumpsuit': 1, 'fun': 1, 'flirty':...
4        {'shirt': 2, 'flattering': 1, 'due': 1, 'adjus...
5        {'love': 2, 'tracy': 1, 'reese': 1, 'dresses':...
6        {'aded': 1, 'basket': 1, 'hte': 4, 'last': 1, ...
7        {'ordered': 1, 'carbon': 1, 'store': 1, 'pick'...
8        {'love': 1, 'dress': 1, 'usually': 1, 'get': 1...
9        {'im': 1, '55': 1, '125': 1, 'lbs': 1, 'ordere...
10       {'dress': 2, 'runs': 2, 'small': 1, 'esp': 1, ...
11       {'dress': 1, 'perfection': 1, 'pretty': 1, 'fl...
12       {'find': 1, 'reliant': 1, 'reviews': 1, 'writt...
13       {'bought': 1, 'black': 1, 'xs': 2, 'go': 1, 'l...
14       {'nice': 1, 'choice': 1, 'holiday': 1, 'gather...
15       {'took': 1, 'package': 1, 'wanted': 1, 'fit': ...
16       {'material': 1, 'color': 1, 'nice': 1, 'leg'